# Model recovery attack: `drive-diagnosis`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = np.loadtxt("../datasets/drive-diagnosis.txt")
np.random.shuffle(xy)
x = xy[:,0:-1]
y = xy[:,-1]

min_values = np.array([min(x[:,i]) for i in range(x.shape[1])])
max_values = np.array([max(x[:,i]) for i in range(x.shape[1])])
x = (x-min_values)/(max_values - min_values)
y = y.reshape((len(x), 1)).astype("float32")-1
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-03-01 10:25:37.211450: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-01 10:25:37.211609: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpd1
2022-03-01 10:25:37.211632: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpd1
2022-03-01 10:25:37.211923: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.91.3
2022-03-01 10:25:37.212011: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.91.3
2022-03-01 10:25:37.212030: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.91.3
2022-03-01 10:25:37.275124: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perfor

In [3]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(256, activation="sigmoid")(xin)
    x = tf.keras.layers.Dense(256, activation="sigmoid")(x)
    x = tf.keras.layers.Dense(256, activation="sigmoid")(x)
    output = tf.keras.layers.Dense(256, activation="sigmoid")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    # x = tf.keras.layers.Dense(256, activation="relu")(xin)
    # x = tf.keras.layers.Dense(256, activation="relu")(x)
    output = tf.keras.layers.Dense(11, activation="softmax")(xin)
    return tf.keras.Model(xin, output)

In [4]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.sparse_categorical_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=1000
    )

Iteration 0001: Training loss: 2.5111
Iteration 1001: Training loss: 0.9156
Iteration 2001: Training loss: 0.8125
Iteration 3001: Training loss: 0.9116
Iteration 4001: Training loss: 0.8288
Iteration 5001: Training loss: 0.6053
Iteration 6001: Training loss: 0.8118
Iteration 7001: Training loss: 0.8654
Iteration 8001: Training loss: 0.6315
Iteration 9001: Training loss: 0.8227


In [8]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(1024, activation=act)(xin)
    x = tf.keras.layers.Dense(512, activation=act)(x)
    # x = tf.keras.layers.Dense(256, activation=act)(x)
    # x = tf.keras.layers.Dense(128, activation=act)(x)
    # x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(48, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [7]:
log_attack = mra.attack(
    attack_iter=5,
    make_generator=make_generator,
    input_noise=False,
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

tf.Tensor(2.9479583e-07, shape=(), dtype=float32)
Iteration 0001: RG-uniform: 0.1831, RG-normal: 0.1615, reconstruction validation: 0.1402


KeyboardInterrupt: 

In [8]:
np.savetxt("./log/drive-4.csv", log_attack, delimiter=",")